In [ ]:
import os
import pandas as pd
import numpy as np
import math
from scipy import stats
from enum import Enum

# Define active sections in the hospital
class SectionType(Enum):
    OUTSIDE = "OUTSIDE"
    PRE_SURGERY = "PRE_SURGERY"
    EMERGENCY = "EMERGENCY"
    LABRATORY = "LABRATORY"
    OPERATING_ROOMS = "OPERATING_ROOMS"
    WARD = "WARD"
    ICU = "ICU"
    CCU = "CCU"
    RIP = "RIP"

def get_active_sections():
    return [sec for sec in SectionType if sec not in (SectionType.OUTSIDE, SectionType.RIP)]

# Statistical parameters
alpha = 0.05

# Step 1: Data storage
means_time_in_system = []
means_queue_length = {sec.value: [] for sec in get_active_sections()}
max_queue_length = {sec.value: [] for sec in get_active_sections()}
means_wait_time = {sec.value: [] for sec in get_active_sections()}
max_wait_time = {sec.value: [] for sec in get_active_sections()}
means_awet = []
means_re_surgery = []
means_occupancy = {sec.value: [] for sec in get_active_sections()}
means_prob_full = []

for i in range(1, 6):  # Loop through result_1 to result_5
    folder = f"result{i}"
    patients_df = pd.read_excel(os.path.join(folder, 'patients.xlsx'))

    # --- 1. Average Time of Stay in the System ---
    times = patients_df["total_time_in_system"].dropna().values
    if len(times) > 0:
        means_time_in_system.append(np.mean(times))

    # --- 2. Probability of Emergency Queue Being Full ---
    emergency_queue_capacity = 10
    try:
        emergency_queue_df = pd.read_excel(os.path.join(folder, 'EMERGENCY queue.xlsx'), index_col=0)
        queue_lengths = emergency_queue_df.iloc[:, 0].dropna().values
        if len(queue_lengths) > 0:
            full_count = np.sum(queue_lengths >= emergency_queue_capacity)
            means_prob_full.append(full_count / len(queue_lengths))
    except Exception:
        pass

    # --- 3. Queue Length and Waiting Time per Section ---
    for sec in get_active_sections():
        sec_name = sec.value

        # Extract waiting times
        wait_col = f"{sec_name} Queue Duration"
        if wait_col in patients_df.columns:
            wait_times = patients_df[wait_col].dropna().values
            if len(wait_times) > 0:
                means_wait_time[sec_name].append(np.mean(wait_times))
                max_wait_time[sec_name].append(np.max(wait_times))

        # Extract queue length
        queue_file = os.path.join(folder, f'{sec_name} queue.xlsx')
        try:
            queue_df = pd.read_excel(queue_file, index_col=0)
            queue_values = queue_df.iloc[:, 0].dropna().values
            if len(queue_values) > 0:
                means_queue_length[sec_name].append(np.mean(queue_values))
                max_queue_length[sec_name].append(np.max(queue_values))
        except Exception:
            pass

    # --- 4. Average Number of Re-Surgeries for Complex Patients ---
    complex_patients = patients_df[patients_df["Surgery Type"].str.lower().apply(lambda x: "complicated" in x)]
    if not complex_patients.empty:
        means_re_surgery.append(complex_patients["re-surgery times"].mean())

    # --- 5. Average Bed Utilization in Each Section ---
    for sec in get_active_sections():
        sec_name = sec.value
        entity_file = os.path.join(folder, f'{sec_name} entity.xlsx')
        try:
            entity_df = pd.read_excel(entity_file, index_col=0)
            occupancy_values = entity_df.iloc[:, 0].dropna().values
            if len(occupancy_values) > 0:
                means_occupancy[sec_name].append(np.mean(occupancy_values))
        except Exception:
            pass

    # --- 6. AWET ---
    emergency_patients = patients_df[patients_df["Patient Type"].str.lower() == "nonelective"]
    if not emergency_patients.empty:
        wait_until_surgery_begins = (
            emergency_patients["EMERGENCY Queue Duration"] +
            emergency_patients["EMERGENCY Serving Duration"] +
            emergency_patients["LABRATORY Serving Duration"] +
            emergency_patients["OPERATING_ROOMS Queue Duration"]
        )
        means_awet.append(wait_until_surgery_begins.mean(skipna=True))

# Function to compute mean and confidence intervals
def compute_stats(data):
    data = np.array(data)
    data = data[~np.isnan(data)]  # Remove NaNs
    n = len(data)
    if n > 1:
        mean_val = np.mean(data)
        se = np.std(data, ddof=1) / math.sqrt(n)
        t_critical = stats.t.ppf(1 - alpha / 2, df=n - 1)
        ci = (mean_val - t_critical * se, mean_val + t_critical * se)
    else:
        mean_val, ci = np.nan, (np.nan, np.nan)
    return mean_val, ci

# --- 1. Average Time of Stay in the System ---
mean_time, ci_time = compute_stats(means_time_in_system)
print("\n1. Average Time of Stay in the System:")
print(f"   Mean = {mean_time:.4f} (CI: {ci_time[0]:.4f} to {ci_time[1]:.4f})")

# --- 2. Probability of Emergency Queue Being Full ---
mean_prob, ci_prob = compute_stats(means_prob_full)
print("\n2. Probability of Emergency Queue Being Full:")
print(f"   Mean = {mean_prob:.4f} (CI: {ci_prob[0]:.4f} to {ci_prob[1]:.4f})")

# --- 3. Queue Length and Waiting Time per Section ---
print("\n3. Queue Length and Waiting Time per Section:")
for sec_name in means_queue_length.keys():

    mean_queue, ci_queue = compute_stats(means_queue_length[sec_name])
    mean_max_queue, ci_max_queue = compute_stats(max_queue_length[sec_name])  # Confidence Interval for Max Queue
    mean_wait, ci_wait = compute_stats(means_wait_time[sec_name])
    mean_max_wait, ci_max_wait = compute_stats(max_wait_time[sec_name])  # Confidence Interval for Max Wait

    print(f"\nSection: {sec_name}")
    print(f"   Average Waiting Time = {mean_wait:.4f} (CI: {ci_wait[0]:.4f} to {ci_wait[1]:.4f})")
    print(f"   Average Maximum Waiting Time = {mean_max_wait:.4f} (CI: {ci_max_wait[0]:.4f} to {ci_max_wait[1]:.4f})")
    print(f"   Average Queue Length = {mean_queue:.4f} (CI: {ci_queue[0]:.4f} to {ci_queue[1]:.4f})")
    print(f"   Average Maximum Queue Length = {mean_max_queue:.4f} (CI: {ci_max_queue[0]:.4f} to {ci_max_queue[1]:.4f})")

# --- 4. Average Number of Re-Surgeries for Complex Patients ---
mean_re_surgery, ci_re_surgery = compute_stats(means_re_surgery)
print("\n4. Average Number of Re-Surgeries for Patients with Complicated Disease:")
print(f"   Mean = {mean_re_surgery:.4f} (CI: {ci_re_surgery[0]:.4f} to {ci_re_surgery[1]:.4f})")

# --- 5. Average Bed Utilization in Each Section ---
print("\n5. Average Bed Utilization per Section:")
for sec_name in means_occupancy.keys():
    mean_occupancy, ci_occupancy = compute_stats(means_occupancy[sec_name])
    print(f"   Section: {sec_name} - Mean Occupancy = {mean_occupancy:.4f} (CI: {ci_occupancy[0]:.4f} to {ci_occupancy[1]:.4f})")

# --- 6. AWET ---
mean_awet, ci_awet = compute_stats(means_awet)
print("\n6. Average Waiting Time Until Surgery (AWET):")
print(f"   Mean = {mean_awet:.4f} (CI: {ci_awet[0]:.4f} to {ci_awet[1]:.4f})")

In [ ]:
import os
import pandas as pd
import numpy as np
import math
from scipy import stats
from enum import Enum

# Define hospital sections
class SectionType(Enum):
    OUTSIDE = "OUTSIDE"
    PRE_SURGERY = "PRE_SURGERY"
    EMERGENCY = "EMERGENCY"
    LABRATORY = "LABRATORY"
    OPERATING_ROOMS = "OPERATING_ROOMS"
    WARD = "WARD"
    ICU = "ICU"
    CCU = "CCU"
    RIP = "RIP"

def get_active_sections():
    return [sec for sec in SectionType if sec not in (SectionType.OUTSIDE, SectionType.RIP)]

# Confidence level settings
alpha = 0.05
cutoff_time = 250  # Remove first 250 hours

# Data storage
data_A, data_B = {}, {}

metrics = [
    "time_in_system", "prob_full", "re_surgery", "awet"
]
for sec in get_active_sections():
    metrics += [
        f"queue_length_{sec.value}", f"max_queue_length_{sec.value}",
        f"wait_time_{sec.value}", f"max_wait_time_{sec.value}",
        f"occupancy_{sec.value}"
    ]

for metric in metrics:
    data_A[metric] = []
    data_B[metric] = []

# Load and filter data
for i in range(1, 6):  
    for system, data_store in zip(["A", "B"], [data_A, data_B]):
        folder = f"result_{system}_{i}"
        
        # Load Patients Data (No filtering needed)
        patients_df = pd.read_excel(os.path.join(folder, 'patients.xlsx'))

        # Time in system
        data_store["time_in_system"].append(np.mean(patients_df["total_time_in_system"].dropna()))

        # Load and filter Events File
        try:
            events_df = pd.read_excel(os.path.join(folder, 'events.xlsx'))
            events_df = events_df[events_df["Event Time"] > cutoff_time]
        except Exception:
            pass

        # Probability of Emergency Queue Being Full
        emergency_queue_capacity = 10
        try:
            emergency_queue_df = pd.read_excel(os.path.join(folder, 'EMERGENCY queue.xlsx'), index_col=0)
            emergency_queue_df = emergency_queue_df[emergency_queue_df.iloc[:, 0] > cutoff_time]  # Filter by time
            queue_lengths = emergency_queue_df.iloc[:, 1].dropna().values
            if len(queue_lengths) > 0:
                full_count = np.sum(queue_lengths >= emergency_queue_capacity)
                data_store["prob_full"].append(full_count / len(queue_lengths))
        except Exception:
            data_store["prob_full"].append(np.nan)

        # Section-wise filtering
        for sec in get_active_sections():
            sec_name = sec.value

            # Queue Lengths (Filter by time)
            queue_file = os.path.join(folder, f'{sec_name} queue.xlsx')
            try:
                queue_df = pd.read_excel(queue_file, index_col=0)
                queue_df = queue_df[queue_df.iloc[:, 0] > cutoff_time]  # Filter rows
                queue_values = queue_df.iloc[:, 1].dropna().values
                if len(queue_values) > 0:
                    data_store[f"queue_length_{sec_name}"].append(np.mean(queue_values))
                    data_store[f"max_queue_length_{sec_name}"].append(np.max(queue_values))
                else:
                    data_store[f"queue_length_{sec_name}"].append(np.nan)
                    data_store[f"max_queue_length_{sec_name}"].append(np.nan)
            except Exception:
                data_store[f"queue_length_{sec_name}"].append(np.nan)
                data_store[f"max_queue_length_{sec_name}"].append(np.nan)

            # Waiting Time (No filtering needed)
            wait_col = f"{sec_name} Queue Duration"
            if wait_col in patients_df.columns:
                wait_times = patients_df[wait_col].dropna().values
                data_store[f"wait_time_{sec_name}"].append(np.mean(wait_times) if len(wait_times) > 0 else np.nan)
                data_store[f"max_wait_time_{sec_name}"].append(np.max(wait_times) if len(wait_times) > 0 else np.nan)
            else:
                data_store[f"wait_time_{sec_name}"].append(np.nan)
                data_store[f"max_wait_time_{sec_name}"].append(np.nan)

            # Bed Utilization (Filter by time)
            entity_file = os.path.join(folder, f'{sec_name} entity.xlsx')
            try:
                entity_df = pd.read_excel(entity_file, index_col=0)
                entity_df = entity_df[entity_df.iloc[:, 0] > cutoff_time]  # Filter rows
                occupancy_values = entity_df.iloc[:, 1].dropna().values
                data_store[f"occupancy_{sec_name}"].append(np.mean(occupancy_values) if len(occupancy_values) > 0 else np.nan)
            except Exception:
                data_store[f"occupancy_{sec_name}"].append(np.nan)

        # Number of Re-Surgeries
        complex_patients = patients_df[patients_df["Surgery Type"].str.lower().apply(lambda x: "complicated" in x)]
        if not complex_patients.empty:
            data_store["re_surgery"].append(complex_patients["re-surgery times"].mean())
        else:
            data_store["re_surgery"].append(np.nan)

        # AWET Calculation
        emergency_patients = patients_df[patients_df["Patient Type"].str.lower() == "nonelective"]
        if not emergency_patients.empty:
            wait_until_surgery_begins = (
                emergency_patients["EMERGENCY Queue Duration"] +
                emergency_patients["EMERGENCY Serving Duration"] +
                emergency_patients["LABRATORY Serving Duration"] +
                emergency_patients["OPERATING_ROOMS Queue Duration"]
            )
            data_store["awet"].append(wait_until_surgery_begins.mean(skipna=True))
        else:
            data_store["awet"].append(np.nan)

# Function to perform t-test and compute confidence interval
def paired_t_test(data_A, data_B, metric):
    A_values = np.array(data_A[metric])
    B_values = np.array(data_B[metric])

    # Remove NaN pairs
    mask = ~np.isnan(A_values) & ~np.isnan(B_values)
    A_values = A_values[mask]
    B_values = B_values[mask]

    if len(A_values) < 2:
        return np.nan, np.nan, np.nan, np.nan

    differences = A_values - B_values
    mean_diff = np.mean(differences)
    t_stat, p_value = stats.ttest_rel(A_values, B_values)

    # Confidence interval
    se_diff = stats.sem(differences)
    t_critical = stats.t.ppf(1 - alpha/2, df=len(differences)-1)
    ci_lower = mean_diff - t_critical * se_diff
    ci_upper = mean_diff + t_critical * se_diff

    return mean_diff, ci_lower, ci_upper, p_value

# Print results
print("\nPaired T-Test Results for All Parameters (A - B):\n")

for metric in metrics:
    mean_diff, ci_lower, ci_upper, p_value = paired_t_test(data_A, data_B, metric)

    print(f"{metric.replace('_', ' ').title()}:")
    print(f"   Mean Difference = {mean_diff:.4f}")
    print(f"   95% CI: ({ci_lower:.4f} to {ci_upper:.4f})")
    print(f"   p-value = {p_value:.4f}")

    if p_value < alpha:
        print("   Conclusion: Statistically significant difference.\n")
    else:
        print("   Conclusion: No significant difference.\n")